# Introduction

Ground measured solar irradiance data is extremely valuable and critical for benchmarking solar radiation products, modeling climate processes, and understanding the Earth's radiation budget. However, due to high costs and maintenance requirements, there are only a few hundred high-quality stations globally. Partly due to the scarcity, it has historically been difficult to determine if and where there is a nearby solar radiation monitoring station. To address this, this site provides an overview of high-quality solar radiation monitoring stations worldwide and supporting metadata.

A complete list of stations and metadata can be found [here](../station_listing) and downloaded as a csv file [here](https://raw.githubusercontent.com/AssessingSolar/solarstations/main/solarstations.csv).

To find the nearest station to a point of interest, check out the interactive map below. Note that it is possible to click on a station icon to get the station name and country.

```{margin} Map backgrounds
When zooming in on a specific station, consider switching to the satellite image or annual irradiance backgrounds (top right corner).
```

In [4]:
import pandas as pd
import folium
from folium import plugins

df = pd.read_csv('../solarstations.csv').fillna('')

GHIImagery = "https://d2asdkx1wwwi7q.cloudfront.net/v20210621/ghi_global/{z}/z{z}_{x}x{y}.jpg"
DNIImagery = "https://d2asdkx1wwwi7q.cloudfront.net/v20210621/dni_global/{z}/z{z}_{x}x{y}.jpg"
GlobalSolarAtlasAttribution = "Source: GlobalSolarAtlas 2.0. Data from Solargis."
EsriImagery = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
EsriAttribution = "Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community"

# Create Folium map
m = folium.Map(
    location=[0, 15],
    zoom_start=1, min_zoom=1, max_bounds=True,
    control_scale=True,  # Adds distance scale in lower left corner
    tiles='Cartodb Positron')

# Add each station to the map
for index, row in df.iterrows():    
    if (row['Time period'].endswith('-')) or (row['Time period'] == ''):
        color = '#3186cc'  # Blue for active stations
    else:
        color = '#ff422b'  # Read for inactive stations
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Station full name'] + ' - ' + str(row['State']) + ' ' + row['Country'],
        tooltip=row['Abbreviation'],
        radius=5, color=color,
        fill_color=color, fill=True).add_to(m)

folium.raster_layers.TileLayer(EsriImagery, name='World imagery', attr=EsriAttribution, show=False).add_to(m)
folium.raster_layers.TileLayer(GHIImagery, name='GHI', attr=GlobalSolarAtlasAttribution,
                               max_zoom=10, max_native_zoom=10, zoomOffset=1, show=False).add_to(m)
folium.raster_layers.TileLayer(DNIImagery, name='DNI', attr=GlobalSolarAtlasAttribution,
                               max_zoom=10, max_native_zoom=10, zoomOffset=1, show=False).add_to(m)
folium.LayerControl(position='topright').add_to(m)

# Additional options and plugins
# Note it's not possible to change the position of the scale
#plugins.MiniMap(toggle_display=True, zoom_level_fixed=1, minimized=True, position='bottomright').add_to(m)  # Add minimap to the map
plugins.Fullscreen(position='bottomright').add_to(m)  # Add full screen button to map
folium.LatLngPopup().add_to(m)  # Show latitude/longitude when clicking on the map
# plugins.LocateControl(position='topright').add_to(m)  # Add button for your position
# plugins.MeasureControl(position='topleft').add_to(m)  # Add distance length measurement tool

# Create legend
legend_html = """
<div style="position:fixed;
     top: 10px; 
     left: 10px; 
     width: 120px; 
     height: 65px; 
     border:2px solid grey; 
     z-index: 9999;
     background-color:#f2efe9;
     font-size:14px;">
     &nbsp;<b>Station markers</b><br>
     &nbsp;<i class="fa fa-circle-o fa-1x" style="color:#3186cc"></i>&nbsp;Active<br>
     &nbsp;<i class="fa fa-circle-o fa-1x" style="color:#FA8072"></i>&nbsp;Inactive<br>
</div>"""
m.get_root().html.add_child(folium.Element(legend_html))  # Add Legend to map

# Show the map
m

## Acknowledgements
Solar radiation map backgrounds were obtained from the Global Solar Atlas 2.0, a free, web-based application developed and operated by Solargis s.r.o. on behalf of the World Bank Group, utilizing Solargis data, with funding provided by the Energy Sector Management Assistance Program (ESMAP). For additional information see https://globalsolaratlas.info.